In [1]:
import seaborn as sns
import datetime
import pickle
import pandas as pd

In [2]:
station = 'Alexanderplatz'

In [3]:
from ticket_control.model_preprocessing import *

In [4]:
from ticket_control.utils import bezirke

In [5]:
get_station_latlon(station)

array([['Mitte', 52.521388888888886, 13.412638888888887]], dtype=object)

In [12]:
X_pred = pd.DataFrame({
    'date': [datetime.datetime.now()],
    'station name': [station]
})
X_pred = X_pred.set_index('date')
X_pred[['area', 'latitude', 'longitude']] = get_station_latlon(station)
X_pred = get_time(X_pred)
X_pred = cyclic_features(X_pred)
print(X_pred)
X_pred = add_holiday(X_pred)
X_pred = covid(X_pred)
X_pred = get_bezirke(X_pred)
# OneHotEncoding Bezirke
encoder=OneHotEncoder(sparse = False, categories=[list(bezirke.keys())])
X_pred[encoder.get_feature_names_out()] = encoder.fit_transform(X_pred[["bezirk"]])
X_pred[["local_0", "local_1", "local_2"]] = 5
# drpping columns
X_pred = X_pred.drop(columns = ['station name', 'area', 'bezirk', 'month', 'weekday', 'hour'])


                              station name   area   latitude  longitude  year  \
date                                                                            
2023-09-05 15:57:57.522576  Alexanderplatz  Mitte  52.521389  13.412639  2023   

                            month  weekday  hour  hour_sin  hour_cos  \
date                                                                   
2023-09-05 15:57:57.522576      9        1    15 -0.707107 -0.707107   

                            weekday_sin  weekday_cos  month_sin     month_cos  
date                                                                           
2023-09-05 15:57:57.522576     0.781831      0.62349       -1.0 -1.836970e-16  


/Users/annushkak/.pyenv/versions/3.10.6/envs/bvg-project/lib/python3.10/site-packages/sklearn/preprocessing/_encoders.py:868: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(


In [13]:
X_pred.columns

Index(['latitude', 'longitude', 'year', 'hour_sin', 'hour_cos', 'weekday_sin',
       'weekday_cos', 'month_sin', 'month_cos', 'holiday', 'covid',
       'bezirk_Charlottenburg-Wilmersdorf', 'bezirk_Friedrichshain-Kreuzberg',
       'bezirk_Lichtenberg', 'bezirk_Marzahn-Hellersdorf', 'bezirk_Mitte',
       'bezirk_Neukölln', 'bezirk_Pankow', 'bezirk_Reinickendorf',
       'bezirk_Spandau', 'bezirk_Steglitz-Zehlendorf',
       'bezirk_Tempelhof-Schöneberg', 'bezirk_Treptow-Köpenick',
       'bezirk_Brandenburg', 'local_0', 'local_1', 'local_2'],
      dtype='object')

In [17]:
X_pred

,latitude,longitude,year,hour_sin,hour_cos,weekday_sin,weekday_cos,month_sin,month_cos,holiday,...,bezirk_Pankow,bezirk_Reinickendorf,bezirk_Spandau,bezirk_Steglitz-Zehlendorf,bezirk_Tempelhof-Schöneberg,bezirk_Treptow-Köpenick,bezirk_Brandenburg,local_0,local_1,local_2
date,,,,,,,,,,,,,,,,,,,,,
2023-09-05 15:57:57.522576,52.521389,13.412639,2023,-0.707107,-0.707107,0.781831,0.62349,-1.0,-1.836970e-16,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,5,5,5


In [14]:
model = pickle.load(open('../ticket_control/model.pkl', 'rb'))

In [15]:
model.feature_names_in_

array(['latitude', 'longitude', 'year', 'hour_sin', 'hour_cos',
       'weekday_sin', 'weekday_cos', 'month_sin', 'month_cos', 'holiday',
       'covid', 'local_0', 'local_1', 'local_2', 'bezirk_Brandenburg',
       'bezirk_Charlottenburg-Wilmersdorf',
       'bezirk_Friedrichshain-Kreuzberg', 'bezirk_Lichtenberg',
       'bezirk_Marzahn-Hellersdorf', 'bezirk_Mitte', 'bezirk_Neukölln',
       'bezirk_Pankow', 'bezirk_Reinickendorf', 'bezirk_Spandau',
       'bezirk_Steglitz-Zehlendorf', 'bezirk_Tempelhof-Schöneberg',
       'bezirk_Treptow-Köpenick'], dtype=object)

In [16]:
model.predict(X_pred[model.feature_names_in_])

array([1])